# End of week 1 exercise

## Dynamically pick an LLM provider to let MathXpert answer your math questions.

In [ ]:
import os
from enum import StrEnum
from getpass import getpass

from dotenv import load_dotenv
from openai import OpenAI
import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown, Latex

load_dotenv()

## Free Cloud Providers

Grab your free API Keys from these generous sites:

- https://openrouter.ai/
- https://ollama.com/

In [ ]:
class Provider(StrEnum):
    OLLAMA = 'Ollama'
    OPENROUTER = 'OpenRouter'

models: dict[Provider, str] = {
    Provider.OLLAMA: 'gpt-oss:120b-cloud',
    Provider.OPENROUTER: 'qwen/qwen3-4b:free'
}

def get_api_key(env_name: str) -> str:
    '''Gets the value from the environment, otherwise ask the user for it if not set'''
    key = os.environ.get(env_name)
    if not key:
        key = getpass(f'Enter {env_name}:').strip()

    if key:
        print(f'✅ {env_name} provided')
    else:
        print(f'❌ {env_name} provided')
    return key


providers: dict[Provider, OpenAI] = {}

if api_key := get_api_key('OLLAMA_API_KEY'):
    providers[Provider.OLLAMA] = OpenAI(base_url='https://ollama.com/v1', api_key=api_key)

if api_key := get_api_key('OPENROUTER_API_KEY'):
    providers[Provider.OPENROUTER] = OpenAI(base_url='https://openrouter.ai/api/v1', api_key=api_key)

In [ ]:
def get_messages(question: str) -> list[dict[str, str]]:
    """Generate messages for the chat models."""

    system_prompt = '''
    You are MathXpert, an expert Mathematician who makes math fun to learn by relating concepts to real 
    practical usage to whip up the interest in learners.
    
    Explain step-by-step thoroughly how to solve a math problem. Respond in **LaTex**'
    '''

    return [
        {'role': 'system', 'content': system_prompt },
        {'role': 'user', 'content': question},
    ]

In [ ]:
get_messages('Explain how to solve a differentiation problem')

In [ ]:
selected_provider, client = next(iter(providers.items()))

def on_provider_change(change):
    global selected_provider, client

    selected_provider = change['new']
    client = providers.get(selected_provider)


provider_selector = widgets.Dropdown(
    options=list(providers.keys()),
    description='Select an LLM provider:',
    style={'description_width': 'initial'},
)

provider_selector.observe(on_provider_change, names='value')

In [ ]:
handle = display(None, display_id=True)

def ask(client: OpenAI, model: str, question: str):
    try:
        prompt = get_messages(question=question)
        response = client.chat.completions.create(
            model=model,
            messages=prompt,
            stream=True,
        )
    
        output = ''
        for chunk in response:
            output  += chunk.choices[0].delta.content or ''
    
            handle.update(Latex(output))
    except Exception as e:
        clear_output(wait=True) 
        print(f'🔥 An error occurred: {e}')

In [ ]:
display(provider_selector)

In [ ]:
input_label = "Ask your question (Type 'q' to quit): "
question = input(input_label)

while question.strip().lower() not in ['quit', 'q']:
    clear_output(wait=True)
    print(selected_provider, models[selected_provider])
    model = models[selected_provider]
    ask(client, model, question)

    question = input(input_label)